In [53]:
import os
os.chdir('/Users/corentin/Documents/M2/BIPODS/Project/iPMCMC/ipmcmc')
%run generate_data.py
%run linear_gaussian_state_model
%run smc.py
%run csmc.py
%run ipmcmc.py

In [32]:

# 4.1. Linear Gaussian State Space Model
from scipy.spatial.transform import Rotation as R
np.random.seed(420)

# Parameters
r = R.from_rotvec(np.array([7*np.pi/10, 3*np.pi/10, np.pi/20]))
rotation_matrix = r.as_dcm()
scaling_matrix = 0.99*np.eye(3)
beta = np.random.dirichlet(np.ones(20)*0.2, 3).transpose()
alpha = scaling_matrix@rotation_matrix
t_max = 50
mu = np.array([0, 1, 1])
start_var = 0.1*np.eye(3)
omega = np.eye(3)
sigma = 0.1*np.eye(20)


n_particles = 100

linear_transition_model = [LinearMu(default_mean=mu, default_cov=start_var)]+[LinearTransitionModel(default_mean=np.zeros(3),default_cov=omega,default_alpha=alpha) for t in range(1, t_max)]
linear_proposals = [LinearMu(default_mean=mu, default_cov=start_var)]+[LinearProposalModel(default_mean=np.zeros(3),default_cov=omega,default_alpha=alpha) for t in range(1, t_max)]
linear_observation_model = [LinearObservationModel(default_mean=np.zeros(20),default_cov=sigma,default_beta=beta) for t in range(0, t_max)]


# If we want to change the parameters
assert np.all(np.linalg.eigvals(start_var) > 0)
assert np.all(np.linalg.eigvals(omega) > 0)
assert np.all(np.linalg.eigvals(sigma) > 0)

l_states, l_observations = linear_gaussian_state_space(t_max=t_max, mu=mu, start_var=start_var, transition_var=omega, noise_var=sigma, transition_coeffs=alpha, observation_coeffs=beta)
display((l_states.shape, l_observations.shape))

l_particles, l_weights, l_ancestors = smc(l_observations, n_particles,
       linear_transition_model, linear_proposals, linear_observation_model)

((50, 3), (50, 20))

100%|██████████| 49/49 [00:01<00:00, 28.09it/s]


In [31]:
n_nodes = 32
n_conditional_nodes = 16
n_steps = 1
init_conditional_traj = np.zeros((n_conditional_nodes, t_max, len(mu)))

for i in range(n_conditional_nodes):
    particles, _, _ = smc(l_observations, n_particles,
       linear_transition_model, linear_proposals, linear_observation_model)
    init_conditional_traj[i] = particles.mean(axis=1)

100%|██████████| 49/49 [00:01<00:00, 30.20it/s]


NameError: name 'ipmcmc' is not defined

In [54]:
a, b, c, d, e = ipmcmc(n_steps, n_nodes, n_conditional_nodes,
                       l_observations, n_particles, init_conditional_traj,
                       linear_proposals, linear_transition_model, linear_observation_model)

100%|██████████| 49/49 [00:01<00:00, 31.20it/s]


In [62]:
l_observations.shape

(50, 20)

In [60]:
e



array([[[0.00478469, 0.00956938, 0.02870813, 0.        , 0.        ,
         0.        , 0.07655502, 0.        , 0.0861244 , 0.        ,
         0.        , 0.        , 0.12440191, 0.13397129, 0.        ,
         0.        , 0.        , 0.0430622 , 0.14832536, 0.03349282,
         0.05263158, 0.01913876, 0.        , 0.        , 0.        ,
         0.        , 0.02392344, 0.        , 0.1291866 , 0.07177033,
         0.01435407, 0.        ],
        [0.        , 0.00961538, 0.02884615, 0.        , 0.        ,
         0.        , 0.07692308, 0.        , 0.08653846, 0.        ,
         0.        , 0.        , 0.125     , 0.13461538, 0.        ,
         0.        , 0.        , 0.04326923, 0.14903846, 0.03365385,
         0.05288462, 0.01923077, 0.        , 0.        , 0.        ,
         0.        , 0.02403846, 0.        , 0.12980769, 0.07211538,
         0.01442308, 0.        ],
        [0.        , 0.00961538, 0.02884615, 0.        , 0.        ,
         0.        , 0.07692308, 0.